In [1]:
import os

from fastai.vision.all import *
from fastai.vision import *

import pandas as pd

### Prepare datasets

In [10]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Define the paths for the good quality and blurry image datasets
good_quality_path = '/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_train_26-04-2024_15-44-44/ground_truth_trichomes_datasets/trichome_dataset_125_good_quality'
blurry_path = '/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_train_26-04-2024_15-44-44/ground_truth_trichomes_datasets/trichome_dataset_125_blurry'

# Define the paths for the output classification datasets
output_path = '/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_train_26-04-2024_15-44-44/ground_truth_trichomes_datasets/blurry_classification_datasets'

# Classes in the dataset
classes = ['clear', 'cloudy', 'amber']

# Function to split and copy images
def split_and_copy_images(source_path, output_base_path, class_name, quality):
    files = os.listdir(source_path)
    
    # Skip if no files in the folder
    if not files:
        print(f"No files found in {source_path}, skipping...")
        return
    
    train_files, test_files = train_test_split(files, test_size=0.2, random_state=42)

    # Define the train and test output directories
    train_output_path = os.path.join(output_base_path, class_name, 'train', quality)
    test_output_path = os.path.join(output_base_path, class_name, 'test', quality)

    # Create the train and test directories if they don't exist
    os.makedirs(train_output_path, exist_ok=True)
    os.makedirs(test_output_path, exist_ok=True)

    for file in train_files:
        src_file_path = os.path.join(source_path, file)
        if os.path.isfile(src_file_path):
            shutil.copy(src_file_path, os.path.join(train_output_path, file))

    for file in test_files:
        src_file_path = os.path.join(source_path, file)
        if os.path.isfile(src_file_path):
            shutil.copy(src_file_path, os.path.join(test_output_path, file))

# Process each class for both good quality and blurry datasets
for cls in classes:
    good_quality_class_path = os.path.join(good_quality_path, cls)
    blurry_class_path = os.path.join(blurry_path, cls)
    
    split_and_copy_images(good_quality_class_path, output_path, cls, 'good_quality')
    split_and_copy_images(blurry_class_path, output_path, cls, 'blurry')

# Print the paths of the output classification datasets
for cls in classes:
    print(f"{cls.capitalize()} dataset paths:")
    print(f"  Train/Good Quality: {os.path.join(output_path, cls, 'train', 'good_quality')}")
    print(f"  Train/Blurry: {os.path.join(output_path, cls, 'train', 'blurry')}")
    print(f"  Test/Good Quality: {os.path.join(output_path, cls, 'test', 'good_quality')}")
    print(f"  Test/Blurry: {os.path.join(output_path, cls, 'test', 'blurry')}")


Clear dataset paths:
  Train/Good Quality: /home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_train_26-04-2024_15-44-44/ground_truth_trichomes_datasets/blurry_classification_datasets/clear/train/good_quality
  Train/Blurry: /home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_train_26-04-2024_15-44-44/ground_truth_trichomes_datasets/blurry_classification_datasets/clear/train/blurry
  Test/Good Quality: /home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_train_26-04-2024_15-44-44/ground_truth_trichomes_datasets/blurry_classification_datasets/clear/test/good_quality
  Test/Blurry: /home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_train_26-04-2024_15-44-44/ground_truth_trichomes_datasets/blurry_classification_datasets/clear/test/blurry
Cloudy dataset paths:
  Train/Good Quality: /home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_train_26-04-2024_15-44-44/ground_truth_trichomes_datasets/blurry_classific

## Image Classification for each of the classes (clear, cloudy and amber) to good/blurry classes

In [26]:
clear_dataset_path = "/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_train_26-04-2024_15-44-44/ground_truth_trichomes_datasets/blurry_classification_datasets/clear/train"
cloudy_dataset_path = "/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_train_26-04-2024_15-44-44/ground_truth_trichomes_datasets/blurry_classification_datasets/cloudy/train"
amber_dataset_path = "/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_train_26-04-2024_15-44-44/ground_truth_trichomes_datasets/blurry_classification_datasets/amber/train"

In [17]:
# models to compare 
available_models = {
    'alexnet': models.alexnet,
    'resnet34': models.resnet34,
    'resnet50': models.resnet50,
    'resnet101': models.resnet101,
    'resnet152': models.resnet152,
    'resnext50_32x4d': models.resnext50_32x4d,
    'resnext101_32x8d': models.resnext101_32x8d,
    'wide_resnet50_2': models.wide_resnet50_2,
    'wide_resnet101_2': models.wide_resnet101_2,
    'vgg16_bn': models.vgg16_bn,
    'vgg19_bn': models.vgg19_bn,
    'squeezenet1_1': models.squeezenet1_1,
    'densenet121': models.densenet121,
    'densenet169': models.densenet169,
    'densenet201': models.densenet201,
}

In [25]:
from fastai.vision.all import *
import PIL

# Custom Transform to Resize with Padding
def custom_transform(size):
    return Resize(size, method='pad', pad_mode='zeros')

# Custom Transform to HSV using fastai's rgb2hsv
class RGB2HSV(Transform):
    def encodes(self, img: PILImage): 
        return rgb2hsv(img)

def get_dataloaders(dataset_path):
    # Define all DataLoaders configurations
    dataloaders_dict = {
        "raw": ImageDataLoaders.from_folder(
            path=dataset_path,
            item_tfms=Resize(128),
            bs=16,
            valid_pct=0.25
        ),
        
        "resize_with_padding": ImageDataLoaders.from_folder(
            path=dataset_path,
            item_tfms=custom_transform(128),  # Adjust size as needed
            batch_tfms=aug_transforms(size=128),  # Apply data augmentation
            bs=16,
            valid_pct=0.25
        ),
        
        "convert_to_hsv": ImageDataLoaders.from_folder(
            path=dataset_path,
            item_tfms=Resize(128),  # Resize before applying HSV transform
            batch_tfms=[RGB2HSV(), *aug_transforms(size=128)],
            bs=16,
            valid_pct=0.25
        ),
        
        "normalize_pixels": ImageDataLoaders.from_folder(
            path=dataset_path,
            item_tfms=Resize(128),  # Resize before applying HSV transform
            batch_tfms=[Normalize.from_stats(*imagenet_stats), *aug_transforms(size=128)],
            bs=16,
            valid_pct=0.25
        ),
        
        "brightness_contrast": ImageDataLoaders.from_folder(
            path=dataset_path,
            item_tfms=Resize(128), 
            batch_tfms=[Brightness(max_lighting=0.2, p=0.75), Contrast(max_lighting=0.2, p=0.75), *aug_transforms(size=128)],
            bs=16,
            valid_pct=0.25
        ),
        
        "combined_resize_padding_hsv": ImageDataLoaders.from_folder(
        path=dataset_path,
        item_tfms=custom_transform(size=128),  # Resize and HSV transform
        batch_tfms=[
            RGB2HSV(),
            *aug_transforms(size=128, flip_vert=True, max_rotate=10)
        ],
        bs=16,
        valid_pct=0.25
        ),
        
        "combined_hsv_normalize": ImageDataLoaders.from_folder(
            path=dataset_path,
            item_tfms=Resize(128),  # Resize before applying HSV transform
            batch_tfms=[
                RGB2HSV(),
                Normalize.from_stats(*imagenet_stats),
                *aug_transforms(size=128)
            ],
            bs=16,
            valid_pct=0.25
        ),
        
        "combined_padding_normalize": ImageDataLoaders.from_folder(
            path=dataset_path,
            item_tfms=custom_transform(128),  # Resize with padding
            batch_tfms=[
                Normalize.from_stats(*imagenet_stats),
                *aug_transforms(size=128)
            ],
            bs=16,
            valid_pct=0.25
        ),
        
        "combined_padding_brightness_contrast": ImageDataLoaders.from_folder(
            path=dataset_path,
            item_tfms=custom_transform(128),  # Resize with padding
            batch_tfms=[
                Brightness(max_lighting=0.2, p=0.75),
                Contrast(max_lighting=0.2, p=0.75),
                *aug_transforms(size=128)
            ],
            bs=16,
            valid_pct=0.25
        ),
        
        "combined_hsv_brightness_contrast": ImageDataLoaders.from_folder(
            path=dataset_path,
            item_tfms=Resize(128),  # Resize before applying HSV transform
            batch_tfms=[
                RGB2HSV(),
                Brightness(max_lighting=0.2, p=0.75),
                Contrast(max_lighting=0.2, p=0.75),
                *aug_transforms(size=128)
            ],
            bs=16,
            valid_pct=0.25
        ),
        
        "combined_all": ImageDataLoaders.from_folder(
            path=dataset_path,
            item_tfms=custom_transform(size=128),  # Resize and HSV transform
            batch_tfms=[
                RGB2HSV(),
                *aug_transforms(size=128, flip_vert=True, max_rotate=10),
                Brightness(max_lighting=0.2, p=0.75),
                Contrast(max_lighting=0.2, p=0.75),
            ],
            bs=16,
            valid_pct=0.25
        )
    }
    
    return dataloaders_dict

In [27]:
clear_dataloaders_dict = get_dataloaders(clear_dataset_path)
cloudy_dataloaders_dict = get_dataloaders(cloudy_dataset_path)
amber_dataloaders_dict = get_dataloaders(amber_dataset_path)

In [28]:
def train_model(dataloaders_dict, model_name):
    from fastai.metrics import Precision, Recall, RocAucMulti
    from fastai.vision.all import vision_learner

    # Define the model architecture and metrics
    precision_macro = Precision(average='macro')
    recall_macro = Recall(average='macro')

    results = {}

    for name, dls in dataloaders_dict.items():
        print(f"Training with configuration: {name}")

        model = vision_learner(
            dls=dls,
            arch=available_models[model_name],
            metrics=[error_rate, precision_macro, recall_macro]
        )

        # Train the model
        model.fine_tune(epochs=25)

        # Get and store the results
        results[name] = model.validate()

    # Display the results
    for config, result in results.items():
        print(f"Results for {config}: {result}")

#### Clear Classification

In [ ]:
train_model(cloudy_dataloaders_dict, 'alexnet')

#### Cloudy Classification

In [29]:
train_model(cloudy_dataloaders_dict, 'alexnet')

Training with configuration: raw


epoch,train_loss,valid_loss,error_rate,precision_score,recall_score,time
0,0.770577,0.420775,0.184332,0.790130,0.805132,00:01


epoch,train_loss,valid_loss,error_rate,precision_score,recall_score,time
0,0.646205,0.375080,0.142857,0.835480,0.846807,00:00
1,0.479214,0.358247,0.147465,0.833872,0.828912,00:00
2,0.418711,0.393450,0.156682,0.824726,0.814827,00:00
3,0.386068,0.351711,0.138249,0.848482,0.832144,00:01
4,0.388564,0.286871,0.101382,0.884864,0.884864,00:00
5,0.338930,0.346998,0.138249,0.842996,0.842996,00:01
6,0.316636,0.397245,0.152074,0.825501,0.850810,00:00
7,0.315583,0.423985,0.184332,0.798792,0.834073,00:01
8,0.306058,0.291928,0.119816,0.873377,0.849460,00:00
9,0.235957,0.269163,0.101382,0.880519,0.895717,00:01


Training with configuration: resize_with_padding


epoch,train_loss,valid_loss,error_rate,precision_score,recall_score,time
0,0.971840,0.505535,0.184332,0.809365,0.819540,00:01


...

epoch,train_loss,valid_loss,error_rate,precision_score,recall_score,time
0,0.615255,0.376389,0.175115,0.825603,0.859906,00:01
1,0.613327,0.371660,0.161290,0.838005,0.873615,00:01
2,0.583712,0.331407,0.129032,0.856183,0.885681,00:01
3,0.573963,0.283737,0.115207,0.868790,0.893099,00:01
4,0.573886,0.272058,0.142857,0.843922,0.875117,00:01
5,0.506857,0.272823,0.124424,0.860435,0.889202,00:01
6,0.486146,0.272750,0.129032,0.855256,0.863662,00:01
7,0.454151,0.257020,0.124424,0.862603,0.895493,00:01
8,0.424357,0.237636,0.092166,0.902566,0.891831,00:01
9,0.415796,0.282097,0.129032,0.854253,0.869953,00:01


Training with configuration: combined_all


epoch,train_loss,valid_loss,error_rate,precision_score,recall_score,time
0,0.934131,0.441346,0.161290,0.822955,0.845558,00:01


epoch,train_loss,valid_loss,error_rate,precision_score,recall_score,time
0,0.703545,0.457146,0.156682,0.826505,0.833940,00:01
1,0.614896,0.457471,0.170507,0.814117,0.808137,00:01
2,0.561174,0.451535,0.142857,0.848854,0.832447,00:01
3,0.571368,0.457916,0.175115,0.807030,0.822788,00:01
4,0.559069,0.472300,0.156682,0.830316,0.821809,00:01
5,0.515618,0.390565,0.147465,0.837069,0.841032,00:01
6,0.494253,0.407128,0.119816,0.874150,0.859276,00:01
7,0.485997,0.446645,0.175115,0.811768,0.837953,00:01
8,0.476915,0.349555,0.124424,0.870104,0.852697,00:01
9,0.419627,0.428656,0.147465,0.835998,0.847098,00:01


Results for raw: [0.3431016504764557, 0.10138248652219772, 0.8895024875621891, 0.8776287864171329]
Results for resize_with_padding: [0.2481851577758789, 0.09677419066429138, 0.8987403100775193, 0.9002210433244916]
Results for convert_to_hsv: [0.33487963676452637, 0.11981566995382309, 0.8716086956521739, 0.8825187969924813]
Results for normalize_pixels: [0.3093346655368805, 0.12442396581172943, 0.8787448347107438, 0.8741071428571429]
Results for brightness_contrast: [0.3234454095363617, 0.13364055752754211, 0.8601811906816221, 0.8677325581395349]
Results for combined_resize_padding_hsv: [0.2671280801296234, 0.09677419066429138, 0.8958897922312556, 0.8978304284676834]
Results for combined_hsv_normalize: [0.19974173605442047, 0.08294931054115295, 0.9150869565217392, 0.9150869565217392]
Results for combined_padding_normalize: [0.26715874671936035, 0.11520737409591675, 0.873139070442992, 0.8791274672569636]
Results for combined_padding_brightness_contrast: [0.31398871541023254, 0.1244239658

In [30]:
train_model(amber_dataloaders_dict, 'alexnet')

Training with configuration: raw


epoch,train_loss,valid_loss,error_rate,precision_score,recall_score,time
0,0.948658,0.564434,0.263158,0.717949,0.702381,00:00


epoch,train_loss,valid_loss,error_rate,precision_score,recall_score,time
0,1.083874,0.641510,0.421053,0.572222,0.577381,00:00
1,0.880680,0.699877,0.473684,0.533333,0.535714,00:00
2,0.795576,0.722316,0.473684,0.533333,0.535714,00:00
3,0.694363,0.756447,0.368421,0.638889,0.648810,00:00
4,0.619543,0.771364,0.315789,0.677778,0.690476,00:00
5,0.543180,0.806875,0.315789,0.677778,0.690476,00:00
6,0.487610,0.887526,0.315789,0.677778,0.690476,00:00
7,0.468328,0.927046,0.315789,0.677778,0.690476,00:00
8,0.438248,0.977014,0.315789,0.677778,0.690476,00:00
9,0.408318,1.019171,0.368421,0.613636,0.619048,00:00


Training with configuration: resize_with_padding


epoch,train_loss,valid_loss,error_rate,precision_score,recall_score,time
0,1.377484,0.838016,0.421053,0.577381,0.572222,00:00


...

epoch,train_loss,valid_loss,error_rate,precision_score,recall_score,time
0,1.173542,1.001471,0.473684,0.538889,0.550000,00:00
1,1.138287,0.886441,0.368421,0.596591,0.621429,00:00
2,1.132499,0.894745,0.421053,0.517857,0.521429,00:00
3,1.042359,0.975784,0.473684,0.488636,0.485714,00:00
4,1.038301,1.055185,0.421053,0.517857,0.521429,00:00
5,0.964648,0.976664,0.315789,0.592857,0.592857,00:00
6,0.923201,0.868625,0.263158,0.650000,0.628571,00:00
7,0.857860,0.788333,0.263158,0.650000,0.628571,00:00
8,0.786355,0.725231,0.263158,0.650000,0.628571,00:00
9,0.734009,0.699166,0.210526,0.728571,0.728571,00:00


Training with configuration: combined_all


epoch,train_loss,valid_loss,error_rate,precision_score,recall_score,time
0,1.126968,1.233943,0.789474,0.198864,0.205556,00:00


epoch,train_loss,valid_loss,error_rate,precision_score,recall_score,time
0,1.047598,1.206375,0.789474,0.198864,0.205556,00:00
1,1.112283,1.146937,0.684211,0.316667,0.316667,00:00
2,1.125484,1.037535,0.578947,0.422222,0.422222,00:00
3,1.132847,0.946727,0.578947,0.422222,0.422222,00:00
4,1.075145,0.875402,0.578947,0.422222,0.422222,00:00
5,1.051640,0.861595,0.368421,0.648810,0.638889,00:00
6,1.025225,0.814194,0.368421,0.648810,0.638889,00:00
7,0.977613,0.715000,0.315789,0.683333,0.683333,00:00
8,0.916081,0.666835,0.368421,0.630682,0.627778,00:00
9,0.879777,0.642956,0.368421,0.630682,0.627778,00:00


Results for raw: [1.0949912071228027, 0.3684210479259491, 0.6136363636363636, 0.6190476190476191]
Results for resize_with_padding: [0.7357688546180725, 0.31578946113586426, 0.6931818181818181, 0.6888888888888889]
Results for convert_to_hsv: [0.6942769289016724, 0.3684210479259491, 0.6477272727272727, 0.6477272727272727]
Results for normalize_pixels: [1.0755382776260376, 0.3684210479259491, 0.6214285714285714, 0.5965909090909092]
Results for brightness_contrast: [0.34183838963508606, 0.10526315867900848, 0.8642857142857143, 0.8642857142857143]
Results for combined_resize_padding_hsv: [0.4228271543979645, 0.15789473056793213, 0.8125, 0.8928571428571428]
Results for combined_hsv_normalize: [0.9134017825126648, 0.2631579041481018, 0.7556818181818181, 0.7556818181818181]
Results for combined_padding_normalize: [0.879795491695404, 0.4736842215061188, 0.4464285714285714, 0.425]
Results for combined_padding_brightness_contrast: [1.917028784751892, 0.42105263471603394, 0.5773809523809523, 0.572